In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt 
%matplotlib qt

## PUNTO 0
Implementar la función
create_gauss_filter (h, w, k_size , para crear filtros gaussianos para
filtrado espectral. Debe retornar un filtro gaussiano de tamaño HxW en dominio espacial y su transformada
de Fourier.

In [2]:
def create_gauss_filter(h, w, k_size, sigma):
    #Creo el filtro gausiano en 1 d
    filtro_gaussiano = cv.getGaussianKernel(k_size, sigma)
    #Lo transformo en 2d con la transpuesta
    filtro_gaussiano = filtro_gaussiano * filtro_gaussiano.T
    #Hago un filtro de todos ceros para padding con las dimensiones de la imagen
    filtro_g_padding = np.zeros((h, w), dtype=np.float32)
    #Coordenadas donde voy a colocar el filtro gaussiano dentro del padding
    start_x = (h - k_size) // 2
    start_y = (w - k_size) // 2
    #Reemplazo los ceros por el filtro gaussiano
    filtro_g_padding[start_x:start_x + k_size, start_y:start_y + k_size] = filtro_gaussiano
    # Calculo la FFT y la shifteo
    t_fourier = np.fft.fft2(filtro_g_padding)
    
    return filtro_g_padding, t_fourier

## PUNTO 1
Graficar ambas representaciones para diferentes tamaños de 
kernel y sigmal.

In [3]:
kernel, kernel_tfs = create_gauss_filter(h=480, w=480, k_size=15, sigma=2)

plt.figure(figsize=(12,12))
plt.subplot(1,2,1)
plt.axis('off')
plt.title("Kernel Gausiano")
plt.imshow(kernel, cmap='jet')
plt.subplot(1,2,2)
plt.axis('off')
plt.title("FFT del kernel")
tf_shift = np.fft.fftshift(kernel_tfs)
plt.imshow(np.abs(tf_shift), cmap='jet')

Aplicar el filtro un imagen para validar el funcionamiento en el dominio espectral.

In [16]:
img = cv.imread('golazo.jpg', cv.IMREAD_GRAYSCALE)
h,w = img.shape
print (h,w)

542 870


In [11]:
kernel, kernel_tf = create_gauss_filter(h, w, k_size=15, sigma=2)

In [18]:
img_fft = np.fft.fft2(img)

In [22]:
filtrada = img_fft * kernel_tf
img_gauss = np.fft.ifftshift(np.real(np.fft.ifft2(filtrada)))

plt.figure(figsize=(8,8))
plt.imshow(img_gauss, cmap='gray')
plt.title('Desenfoque gausiano con enfoque Espectral')

Text(0.5, 1.0, 'Desenfoque gausiano con enfoque Espectral')

## PUNTO 2
Usando el método descripto en el
paper Image Sharpness Measure for Blurred Images in Frequency
Domain” comparar el resultado de un filtrado por convolución con el filtrado espectral

In [21]:
blur_gaussiano = cv.GaussianBlur(img, (15, 15), sigmaX=2)
plt.figure(figsize=(8,8))
plt.imshow(blur_gaussiano, cmap='gray')
plt.title('Blur Convolucional con Gauss')

Text(0.5, 1.0, 'Blur Convolucional con Gauss')

 # Algorithm for image quality measure
Input: Image I of size M×N.
Output: Image Quality measure (FM) where FM stands for Frequency Domain Image Blur Measure
Step 1: Compute F which is the Fourier Transform representation of image I
Step 2: Find Fc which is obtained by shifting the origin of F to centre.
Step 3: Calculate AF = abs (Fc) where AF is the absolute value of the centered Fourier transform of image I.
Step 4: Calculate M = max (AF) where M is the maximum value of the frequency component in F.
Step 5: Calculate TH = the total number of pixels in F whose pixel value > thres, where thres = M/1000.
Step 6: Calculate Image Quality measure (FM) from equat  FM = Th/(MXN)ion (1).

In [26]:
def iqm (image):
    # Step 1: Compute F which is the Fourier Transform representation of image
    F = np.fft.fft2(image)
    
    # Step 2: Find Fc which is obtained by shifting the origin of F to centre.
    Fc = np.fft.fftshift(F)
    
    # Step 3: Calculate AF = abs (Fc) where AF is the absolute value of the centered Fourier transform of image I
    AF = np.abs(Fc)
    
    # Step 4: Calculate M = max (AF) where M is the maximum value of the frequency component in F
    M = np.max(AF)
    
    # Step 5: Calculate TH = the total number of pixels in F whose pixel value > thres, where thres = M/1000.
    thres = M / 1000
    
    # Step 6: Count the number of pixels greater than the threshold
    pix_greater_thres = np.sum(AF > thres)
    
    # Step 7: Calculate the image quality measure (FM) from equation (1).  FM = Th/(MXN)
    FM = pix_greater_thres / (image.shape[0] * image.shape[1])

    return FM    

In [27]:
iqm_convolucion = iqm(blur_gaussiano)
print (iqm_convolucion)

0.018651652033761717


In [29]:
iqm_espectral = iqm (img_gauss)
print(iqm_espectral)

0.018384442465114306


## CONCLUSIÓN PUNTO 2
Viendo que el iqm de la imagen blurreada por convolución y el de la espectral difieren en el cuarto decimal, considero que son equivalentes, sin embargo, estrictamente hablando, un iqm menor indica menores frecuencias altas, o sea mejor efecto blur, por lo que el espectral sería el mejor de los dos.

## PUNTO 3

Repetir
la comparación usando uno de los métodos descriptos en el apéndice del paper “Analysis of
focus measure operators in shape from focus”



In [34]:
def brenner_fm (image):
   
    height, width = image.shape
    focus_measure = 0
    for i in range(height - 2):
        for j in range(width):
            focus_measure += (int(image[i, j]) - int(image[i + 2, j])) ** 2
    
    return focus_measure

In [35]:
brenner_fm (blur_gaussiano)

64485818

In [36]:
brenner_fm (img_gauss)

63917020

## CONCLUSIÓN PUNTO 3Nuevamente la diferencia es de menos de 500 mil aproximadamente en 64 millones, por lo que es de 2 órdenes de magnitud menores. Es más significativa que en el método iqm, pero igual muestra que no hay grandes diferencias entre los dos métodos. Estrictamente hablando, el focus measure de brenner me indica que mientras mayor el valor más sharp es la imagen, por lo que por este método reafirmamos que el blurreo por filtro espectral es el mejor..